In [2]:
import pickle
import re

In [1]:
!pwd

/data/home/advaitmb/notebooks/Saaket/speech-emotion-recognition


In [ ]:
audio = pickle.load('/data/home/advaitmb/notebooks/Saaket/speech-emotion-recognition/text2emotion/')

In [3]:
audiocode2text = pickle.load(open('audiocode2text.pkl', 'rb'))

In [5]:
audiocode2text

{'Ses04F_script02_1_M008': 'No of course our flashlight, yours and mine, mi flashlight es su flashlight naturally.  How could we not think to bring it?',
 'Ses05F_impro06_F012': 'I feel really bad for his family, you know.',
 'Ses02M_script03_2_M046': 'Shut up.',
 'Ses01F_script01_1_M026': "So what.  I'm not fast with women.",
 'Ses01M_script03_1_F012': "The pity is that one can't stay happy.",
 'Ses01F_script01_1_M002': 'Four o clock this morning.  I heard a crack and I woke up and looked out.  He was standing right over there when it cracked.',
 'Ses02M_script01_2_M015': 'Wait, why my above all?',
 'Ses05F_script01_1_M008': "Being dishonest with him.  It's the kind of thing that pays off and now it's paying off.",
 'Ses05M_impro08_M018': 'Okay?  Does it list your IP address there?',
 'Ses05F_script02_1_F004': "I mean you keep saying ,my flashlight, as if it's somehow just yours.  How's that suppose to make me feel?",
 'Ses02F_script03_2_F004': 'Just a little perhaps, nothing serious.

In [18]:
import unicodedata

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [19]:
import pandas as pd
df = pd.read_csv('audio_df_improvised.csv')

df.reset_index(inplace=True)
df.head()

,index,start_time,end_time,wav_file,emotion,val,act,dom
0,0,6.2901,8.2357,Ses01F_impro01_F000,neu,2.5,2.5,2.5
1,1,10.0100,11.3925,Ses01F_impro01_F001,neu,2.5,2.5,2.5
2,2,14.8872,18.0175,Ses01F_impro01_F002,neu,2.5,2.5,2.5
3,3,27.4600,31.4900,Ses01F_impro01_F005,neu,2.5,3.5,2.0
4,4,85.2700,88.0200,Ses01F_impro01_F012,ang,2.0,3.5,3.5


In [32]:
# Prepare text data

text_df = pd.DataFrame()
text_df['wav_file'] = df['wav_file']
text_df['emotion'] = df['emotion']
text_df['transcription'] = [normalizeString(audiocode2text[code]) for code in df['wav_file']]
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
text_df['emotion'] = le.fit(text_df['emotion'])
text_df.to_csv('text_df.csv', index=False)
# text_train = pd.DataFrame()
# text_train['wav_file'] = train['wav_file']
# text_train['emotion'] = train['emotion']
# text_train['transcription'] = [normalizeString(audiocode2text[code]) for code in train['wav_file']]

# text_valid = pd.DataFrame()
# text_valid['wav_file'] = valid['wav_file']
# text_valid['emotion'] = valid['emotion']
# text_valid['transcription'] = [normalizeString(audiocode2text[code]) for code in valid['wav_file']]

# text_train.to_csv('text_train.csv', index=False)
# text_valid.to_csv('text_valid.csv', index=False)

In [26]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
text_df['emotion'] = le.fit(text_df['emotion'])

In [30]:
list(le.inverse_transform([0,1,2,3]))

['ang', 'hap', 'neu', 'sad']

In [40]:

#deal with tensors
import torch   

#handling text data
from torchtext import data 
#Reproducing same results
SEED = 2019

#Torch
torch.manual_seed(SEED)

#Cuda algorithms
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize='spacy',batch_first=True,include_lengths=True)
LABEL = data.LabelField(dtype = torch.float,batch_first=True)

fields = [(None, None),('emotion', LABEL), ('transcription',TEXT)]

In [41]:
# We split the dataframe into train and test segments

from sklearn.model_selection import train_test_split

train, valid = train_test_split(audio_impro_processed, test_size=0.2)

In [42]:
#loading custom dataset
training_data=data.TabularDataset(path = 'text_df.csv',format = 'csv',fields = fields,skip_header = True)

#print preprocessed text
print(vars(training_data.examples[0]))

{'transcription': ['excuse', 'me', '.'], 'emotion': 'LabelEncoder()'}


In [43]:
import random
train_data, valid_data = training_data.split(split_ratio=0.7, random_state = random.seed(SEED))

In [44]:
#initialize glove embeddings
TEXT.build_vocab(train_data,min_freq=3,vectors = "glove.6B.100d")  
LABEL.build_vocab(train_data)

#No. of unique tokens in text
print("Size of TEXT vocabulary:",len(TEXT.vocab))

#No. of unique tokens in label
print("Size of LABEL vocabulary:",len(LABEL.vocab))

#Commonly used words
print(TEXT.vocab.freqs.most_common(10))  

#Word dictionary
print(TEXT.vocab.stoi)  

.vector_cache/glove.6B.zip: 862MB [06:30, 2.21MB/s]                                
100%|█████████▉| 399318/400000 [00:20<00:00, 20303.48it/s]

Size of TEXT vocabulary: 721
Size of LABEL vocabulary: 1
[('.', 2724), ('i', 1417), ('you', 1321), ('to', 730), ('it', 646), ('s', 554), ('and', 520), ('that', 513), ('?', 509), ('know', 501)]
defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7f37e05eb6d8>>, {'night': 479, 'you': 4, 'again': 232, 'student': 489, 'god': 159, 'find': 194, 'month': 664, 'were': 147, 'excuse': 629, 'fifty': 152, 'any': 129, 'guy': 646, 'instead': 544, 'personal': 675, 'for': 31, 'birth': 501, 'if': 63, 'really': 49, 'difficult': 515, 'cemetery': 610, 'malibu': 659, 'type': 579, 'plenty': 559, 'found': 638, 'twenty': 320, 'lots': 409, 'definitely': 234, 'stupid': 572, 'someone': 294, 'yet': 390, 'by': 179, 'company': 616, 'me': 34, 'speak': 700, 'wanted': 302, 'time': 79, 'field': 633, 'least': 466, 'babe': 601, 'went': 211, 'realize': 687, 'yes': 88, 'last': 258, 'but': 32, 'card': 306, 'camping': 608, 'on': 56, 'person': 227, 'miso': 662, 'made': 658, 'house': 459, '

100%|█████████▉| 399318/400000 [00:40<00:00, 20303.48it/s]

In [70]:
#check whether cuda is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  

#set batch size
BATCH_SIZE = 64

#Load an iterator
train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_data, valid_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.transcription),
    sort_within_batch=True,
    device = device)

In [56]:

import torch.nn as nn

class classifier(nn.Module):
    
    #define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout):
        
        #Constructor
        super().__init__()          
        
        #embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        #lstm layer
        self.lstm = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout,
                           batch_first=True)
        
        #dense layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        #activation function
        self.act = nn.Softmax(dim=1)
        
    def forward(self, text, text_lengths):
        
        #text = [batch size,sent_length]
        embedded = self.embedding(text)
        #embedded = [batch size, sent_len, emb dim]
      
        #packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths,batch_first=True)
        
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
        
        #concat the final forward and backward hidden state
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
                
        #hidden = [batch size, hid dim * num directions]
        dense_outputs=self.fc(hidden)

        #Final activation function
        outputs=self.act(dense_outputs)
        
        return outputs

In [57]:
#define hyperparameters
size_of_vocab = len(TEXT.vocab)
embedding_dim = 100
num_hidden_nodes = 32
num_output_nodes = 4
num_layers = 2
bidirection = True
dropout = 0.2

#instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes,num_output_nodes, num_layers, 
                   bidirectional = True, dropout = dropout)

In [58]:
#architecture
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print('The model has {} trainable parameters'.format(count_parameters(model)))

#Initialize the pretrained embedding
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

print(pretrained_embeddings.shape)

classifier(
  (embedding): Embedding(721, 100)
  (lstm): LSTM(100, 32, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (fc): Linear(in_features=64, out_features=4, bias=True)
  (act): Softmax(dim=1)
)
The model has 131752 trainable parameters
torch.Size([721, 100])


In [63]:
import torch.optim as optim

#define optimizer and loss
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

#define metric
def accuracy(preds, y):
    
    return np.mean((preds.argmax(axis=1)==y))


    
#push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [72]:
def train(model, iterator, optimizer, criterion):
    
    #initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    #set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        #resets the gradients after every batch
        optimizer.zero_grad()   
        
        #retrieve text and no. of words
        text, text_lengths = batch.transcription   
        
        #convert to 1D tensor
        predictions = model(text, text_lengths).squeeze()  
        
        #compute the loss
        loss = criterion(predictions, batch.emotion)        
        
        #compute the binary accuracy
        acc = accuracy(predictions, batch.emotion)   
        
        #backpropage the loss and compute the gradients
        loss.backward()       
        
        #update the weights
        optimizer.step()      
        
        #loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [73]:
def evaluate(model, iterator, criterion):
    
    #initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    #deactivating dropout layers
    model.eval()
    
    #deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            #retrieve text and no. of words
            text, text_lengths = batch.transcription
            
            #convert to 1d tensor
            predictions = model(text, text_lengths).squeeze()
            
            #compute loss and accuracy
            loss = criterion(predictions, batch.emotion)
            acc = accuracy(predictions, batch.emotion)
            
            #keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [74]:
N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    #train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    #evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print('\tTrain Loss: {} | Train Acc: {}%'.format(train_loss,train_acc*100))
    print('\t Val. Loss: {} |  Val. Acc: {}%'.format(valid_loss,valid_acc*100 ))

RuntimeError: Expected object of scalar type Long but got scalar type Float for argument #2 'target'

In [75]:
for batch in train_iterator:
    print(batch)


[torchtext.data.batch.Batch of size 64]
	[.emotion]:[torch.cuda.FloatTensor of size 64 (GPU 0)]
	[.transcription]:('[torch.cuda.LongTensor of size 64x2 (GPU 0)]', '[torch.cuda.LongTensor of size 64 (GPU 0)]')

[torchtext.data.batch.Batch of size 64]
	[.emotion]:[torch.cuda.FloatTensor of size 64 (GPU 0)]
	[.transcription]:('[torch.cuda.LongTensor of size 64x8 (GPU 0)]', '[torch.cuda.LongTensor of size 64 (GPU 0)]')

[torchtext.data.batch.Batch of size 64]
	[.emotion]:[torch.cuda.FloatTensor of size 64 (GPU 0)]
	[.transcription]:('[torch.cuda.LongTensor of size 64x9 (GPU 0)]', '[torch.cuda.LongTensor of size 64 (GPU 0)]')

[torchtext.data.batch.Batch of size 64]
	[.emotion]:[torch.cuda.FloatTensor of size 64 (GPU 0)]
	[.transcription]:('[torch.cuda.LongTensor of size 64x13 (GPU 0)]', '[torch.cuda.LongTensor of size 64 (GPU 0)]')

[torchtext.data.batch.Batch of size 64]
	[.emotion]:[torch.cuda.FloatTensor of size 64 (GPU 0)]
	[.transcription]:('[torch.cuda.LongTensor of size 64x11 (GPU 